# Q1
### a.

In [ ]:
import numpy as np
from astropy.timeseries import periodograms

In [31]:
def solve_kepler(e, M):
    """
    Solve Kepler's Equation M = E - e sinE
    """ 
    dEs = []
    E = M + 0.85 * e * np.sign(np.sin(M))
    dE = 1e3
    while abs(E - e * np.sin(E) - M) > 1e-10:
        dE = ((E - e * np.sin(E) - M)) / (1 - e * np.cos(E))
        E -= dE
    return E
        
    

Verify over 0 < e < 1 and 0 < M < 2$\pi$ that M = E - e sin(E)

In [39]:
e = np.random.random(size=10000)
M = 2 * np.pi * np.random.random(size=10000)

for e, M in zip(e, M):
    E = solve_kepler(e, M)
    assert(M - (E - e * np.sin(E)) < 1e-10)

### b.

In [32]:
def true_anomaly(e, M):
    E = solve_kepler(e, M)
    return 2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))

In [ ]:
def RV_model(t, K_star, P, tp, e, omega, gamma):
    M = 2 * np.pi / P * (t - tp)
    f = true_anomaly(e, M)
    return K_star * (np.cos(omega) * np.cos(f) - np.sin(omega) * np.sin(f) + e * np.cos(e)) + gamma


### c.

### d.

In [41]:
!pip install emcee

     |████████████████████████████████| 45 kB 8.2 MB/s  eta 0:00:01


In [ ]:
import pymc3 as pm

In [ ]:
with pm.Model():
    K_star = pm.Uniform('K_star', 0, 50) # max RV
    P = pm.Uniform('P', 0, 50) # days
    tp = pm.Uniform('tp', 0, 100) # day
    e = pm.Uniform('e', 0, 1) # eccentricity
    omega = pm.Uniform('omega', 0, 2 * np.pi) # angle
    gamma = pm.Uniform('gamma', 0, 50) # COM velocity
    

    y = pm.Normal('RV', mu=RV_model(t, K_star, P, tp, e, omega, gamma), sd=y_err, observed=y_obs)

    step = pm.Metropolis()
    traces = pm.sample(draws=25000, tune=1000, step=step, chains=1)

In [ ]:
def traces_to_pandas(traces, burnin):
    varnames = [ var for var in traces.varnames if not var.endswith("_interval__") ]
    cols = { var: traces[var, :burnin] for var in varnames }
    return pd.DataFrame(cols)

df = traces_to_pandas(traces, 1000)

variables = ['K_star', 'P', 'tp', 'e', 'omega', 'gamma']
labels = ['$K_{star}$', '$Period$', '$t_p$', 'eccentricity', '$\omega$', '\gamma']
# limits = [(9.2, 11.2), (2, 12), (45, 55), (0.0, 0.25)] # TODO
# true = [K_star_true, P_true, tp_true, e_true, omega_true, gamma_true]

fig = plt.figure(figsize=(10, 10))
fig.subplots_adjust(bottom=0.1, top=0.95,
                    left=0.1, right=0.95,
                    hspace=0.05, wspace=0.05)

# This function plots multiple panels with the traces
plot_mcmc([df[col] for col in variables],
          labels=labels, limits=limits,
          true_values=true, fig=fig, bins=30, colors='k')

# Plot the model fit
ax = fig.add_axes([0.5, 0.7, 0.45, 0.25])
t_fit = np.linspace(0, 100, 1000) # change to t
y_fit = RV_model(t_fit, **df.median())
# y_true = RV_model(t_fit, K_star_true, P_true, tp_true, e_true, omega_true, gamma_true)

ax.scatter(t, y_obs, s=9, lw=0, c='k')
# ax.plot(t_fit, y_true, '--', c='blue', label="true")
ax.plot(t_fit, y_fit, '-k', label="fit")
ax.legend()
ax.set_xlim(0, 100)
ax.set_xlabel('$t$')
ax.set_ylabel(r'$h_{\rm obs}$')

plt.show()

# Q2

Note, in the write up, $\textbf{r} = \textbf{x}$ and $\dot{\textbf{r}} = \textbf{v}$
### a.
Conservation of angular momentum means that cross product of the position and velocity of an object $\textbf{r}\times\dot{\textbf{r}}$ in orbit is conserved and equal to $|h|\hat{z}$ where $\hat{z} = \hat{r}\times\dot{\hat{r}}$ is perpendicular to the plane of orbit. Thus $\textbf{r(t)}\times\dot{\textbf{r(t)}} = \textbf{r}_0\times\dot{\textbf{r}_0}$ for all t as long as there is no change to the orbit. 

Now to prove the relation:
$$ f\dot{g} - g\dot{f} = 1$$
We know that by applying the f and g equations, we can find $\textbf{r(t)}$ and $\dot{\textbf{r(t)}}$ as a function of $\textbf{r}_0$ and $\dot{\textbf{r}_0}$:


$$
\textbf{r} = f \textbf{r}_0 + g \dot{\textbf{r}_0}\\
\dot{\textbf{r}} = \dot{f} \textbf{r}_0 + \dot{g} \dot{\textbf{r}_0}
$$

Taking the cross products of each side:
$$
\textbf{r} \times \dot{\textbf{r}} = (f \textbf{r}_0 + g \dot{\textbf{r}_0}) \times (\dot{f} \textbf{r}_0 + \dot{g} \dot{\textbf{r}_0})
$$
Using the following two identities for the crossproduct:
$$ \textbf{x} \times \textbf{x} = 0 \\ \textbf{x} \times \textbf{y} = - \textbf{y} \times \textbf{x} $$
We can reduce the RHS to:
$$
\textbf{r} \times \dot{\textbf{r}} = (f \textbf{r}_0 + g \dot{\textbf{r}_0}) \times (\dot{f} \textbf{r}_0 + \dot{g} \dot{\textbf{r}_0}) \\
\textbf{r} \times \dot{\textbf{r}} = (f \textbf{r}_0 \times \dot{g} \dot{\textbf{r}_0}) + (g \dot{\textbf{r}_0} \times \dot{f} \textbf{r}_0) \\
\textbf{r} \times \dot{\textbf{r}} = (f \dot{g} + -g \dot{f}) (\textbf{r}_0 \times \dot{\textbf{r}_0})
$$

However, because of conservation of momentum, $\textbf{r}\times\dot{\textbf{r}} = \textbf{r}_0\times\dot{\textbf{r}_0}$, so we are left with:

$$\fbox{$f\dot{g} - g\dot{f} = 1$}
$$

### b.
The f and g functions are:

$
f = \frac{a}{r_0}(\cos(E - E_0) - 1) + 1 \\
g = (t - t_0) + \frac{1}{n}(\sin(E - E_0) - (E - E_0)) \\
\dot{f} = -\frac{a^2}{rr_0} n \sin(E - E_0) \\
\dot{g} = \frac{a}{r}(\cos(E - E_0) - 1) + 1 \\
$ 


We will set our reference frame so that $E_0 = M_0 = 0$ and that $M = n(t-t_0)$ and also pull out factors of $\frac{a}{r_0}$ and $\frac{a}{r}$ out of $f$ and $\dot{g}$ We can rewrite these equations:


$
f = \frac{a}{r_0}(\cos{E} - 1 + \frac{r_0}{a}) \\
g = \frac{1}{n}(M + \sin{E} - E) \\
\dot{f} = -\frac{a^2}{rr_0} n \sin{E} \\
\dot{g} = \frac{a}{r}(\cos{E} - 1) + \frac{r}{a}) \\
$ 

Now solving for $f\dot{g} - g\dot{f}$:

$f\dot{g} - g\dot{f} \\
= [\frac{a}{r_0}(\cos{E} - 1 + \frac{r_0}{a})][\frac{a}{r}(\cos{E} - 1) + \frac{r}{a})] - [\frac{1}{n}(M + \sin{E} - E)][-\frac{a^2}{rr_0} n \sin{E}] \\
= \frac{a^2}{rr_0}\Big[(\cos{E} - 1 + \frac{r_0}{a})(\cos{E} - 1) + \frac{r}{a}) + (M + \sin{E} - E)\sin{E}\Big] \\
$

Making the following substitutions:

$ M = E - e \sin{E} \\
r = a(1 - e \cos{E}) \\
r_0 = a(1 - e \cos{E_0}) = a(1-e) \\
$

We get:
$f\dot{g} - g\dot{f} \\
= \frac{a^2}{rr_0}\Big[[(\cos{E} - 1 + 1-e)(\cos{E} - 1 + 1 - e \cos{E})] + [(E - e \sin{E} + \sin{E} - E)\sin{E}]\Big] \\
= \frac{a^2}{rr_0}\Big[[(\cos{E} - e)(\cos{E} - e \cos{E})] + [\sin{E}(1 - e)\sin{E}]\Big] \\
= \frac{a^2}{rr_0}\Big[\cos^2{E} - e\cos^2{E} - e \cos{E} + e^2\cos{E} + \sin^2{E} - e\sin^2{E}\Big] \\
= \frac{a^2}{rr_0}\Big[\cos^2{E} + \sin^2{E} - e(\cos^2{E} + \sin^2{E}) - e \cos{E} + e^2\cos{E} \Big] \\
= \frac{a^2}{rr_0}\Big[1 - e - e \cos{E} + e^2\cos{E} \Big] \\
= \frac{a^2}{rr_0}\Big[(1 - e)(1 - e \cos{E})\Big] \\
= \frac{a^2}{rr_0}\frac{rr_0}{a^2} \\
\fbox{$= 1$}
$

Thus if Kepler's equation is satisfied, $f\dot{g} - g\dot{f} = 1$ and therefore angular momentum is conserved